# Preference Alignment with Odds Ratio Preference Optimization (ORPO)

This notebook will guide you through the process of fine-tuning a language model using Odds Ratio Preference Optimization (ORPO). We will use the SmolLM2-135M model which has **not** been through SFT training, so it is not compatible with DPO. This means, you cannot use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
     <h2 style='margin: 0;color:blue'>Exercise: Aligning SmolLM2 with ORPOTrainer</h2>
     <p>Take a dataset from the Hugging Face hub and align a model on it. </p>
     <p><b>Difficulty Levels</b></p>
     <p>🐢 Use the `trl-lib/ultrafeedback_binarized` dataset</p>
     <p>🐕 Try out the `argilla/ultrafeedback-binarized-preferences` dataset</p>
     <p>🦁 Try on a subset of mlabonne's `orpo-dpo-mix-40k` dataset</p>
</div>



## Import libraries


In [1]:
!pip install transformers datasets trl huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.3/318.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.3 MB/s eta 0:00:00


In [2]:
import torch
import os
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format

# Authenticate to Hugging Face
from huggingface_hub import login

login()

## Format dataset

In [3]:
# Load dataset

# TODO: 🦁🐕 change the dataset to one of your choosing
dataset = load_dataset(path="mlabonne/orpo-dpo-mix-40k")
dataset

README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/127M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44245 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'chosen', 'rejected', 'prompt', 'question'],
        num_rows: 44245
    })
})

In [20]:
dataset_train = dataset["train"].select(range(int(dataset["train"].num_rows * 0.2)))
dataset_train

Dataset({
    features: ['source', 'chosen', 'rejected', 'prompt', 'question'],
    num_rows: 8849
})

In [21]:
dataset_test = dataset["train"].select(range(int(dataset["train"].num_rows * 0.8), int(dataset["train"].num_rows * 0.8) + 1000))
dataset_test

Dataset({
    features: ['source', 'chosen', 'rejected', 'prompt', 'question'],
    num_rows: 1000
})

In [ ]:
# TODO: 🐕 If your dataset is not represented as conversation lists, you can use the `process_dataset` function to convert it.

## Define the model

In [22]:
model_name = "HuggingFaceTB/SmolLM2-135M"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32,
).to(device)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
model, tokenizer = setup_chat_format(model, tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-ORPO"
finetune_tags = ["smol-course", "module_1"]

## Train model with ORPO

In [23]:
orpo_args = ORPOConfig(
    # Small learning rate to prevent catastrophic forgetting
    learning_rate=8e-6,
    # Linear learning rate decay over training
    lr_scheduler_type="linear",
    # Maximum combined length of prompt + completion
    max_length=1024,
    # Maximum length for input prompts
    max_prompt_length=512,
    # Controls weight of the odds ratio loss (λ in paper)
    beta=0.1,
    # Batch size for training
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    # Helps with training stability by accumulating gradients before updating
    gradient_accumulation_steps=4,
    # Memory-efficient optimizer for CUDA, falls back to adamw_torch for CPU/MPS
    optim="paged_adamw_8bit" if device == "cuda" else "adamw_torch",
    # Number of training epochs
    num_train_epochs=1,
    # When to run evaluation
    evaluation_strategy="steps",
    # Evaluate every 20% of training
    eval_steps=0.02,
    # Log metrics every step
    logging_steps=5,
    save_steps=5,
    # Gradual learning rate warmup
    warmup_steps=10,
    # Disable external logging
    report_to="none",
    # Where to save model/checkpoints
    output_dir="./results/",
    # Enable MPS (Metal Performance Shaders) if available
    use_mps_device=device == "mps",
    hub_model_id=finetune_name,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    processing_class=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/orpo_trainer.py:275: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/8849 [00:00<?, ? examples/s]

Map:   0%|          | 0/8849 [00:00<?, ? examples/s]

Map:   0%|          | 0/8849 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (8759 > 8192). Running this sequence through the model will result in indexing errors


In [25]:
trainer.train()  # Train the model

# Save the model
trainer.save_model(f"./{finetune_name}")



Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
23,9.013000,1.634753,127.123300,7.866000,3.933000,-0.133623,-0.153027,0.687000,0.019404,-1.530272,-1.336228,15.021619,11.118589,1.574011,-0.607415,0.246889
46,9.372000,1.526719,124.876800,8.008000,4.004000,-0.129403,-0.146497,0.674000,0.017094,-1.464971,-1.294031,14.587958,11.017183,1.465066,-0.616536,0.221354
69,9.033700,1.466018,127.897100,7.819000,3.909000,-0.127739,-0.145787,0.691000,0.018047,-1.457865,-1.277393,14.387379,11.000924,1.405031,-0.609877,0.235695
92,8.822000,1.419691,127.951100,7.815000,3.908000,-0.125567,-0.143277,0.691000,0.017711,-1.432774,-1.255665,13.446980,10.362785,1.358595,-0.610961,0.232993
115,8.152400,1.391505,128.036300,7.810000,3.905000,-0.124404,-0.141823,0.687000,0.017419,-1.418225,-1.244038,12.784728,9.923839,1.330261,-0.612439,0.229837
138,7.896400,1.372055,128.036600,7.810000,3.905000,-0.123368,-0.140619,0.690000,0.017250,-1.406185,-1.233683,12.412391,9.728720,1.310747,-0.613081,0.228037
161,8.491800,1.360674,128.194000,7.801000,3.900000,-0.123139,-0.140201,0.688000,0.017062,-1.402010,-1.231392,11.687284,9.260501,1.299269,-0.614052,0.225715
184,7.570800,1.344437,127.858100,7.821000,3.911000,-0.121720,-0.139183,0.691000,0.017463,-1.391834,-1.217203,11.296266,9.009067,1.283287,-0.611493,0.231783
207,7.560700,1.337805,125.717500,7.954000,3.977000,-0.121336,-0.138457,0.691000,0.017120,-1.384567,-1.213363,10.751367,8.613478,1.276452,-0.613524,0.227333
230,7.712100,1.330254,128.178100,7.802000,3.901000,-0.120700,-0.137703,0.692000,0.017003,-1.377028,-1.207001,10.237239,8.371737,1.268846,-0.614080,0.226085


RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 46608000 vs 46607888

In [26]:
# Save to the huggingface hub if login (HF_TOKEN is set)
trainer.push_to_hub(tags=finetune_tags)

OSError: [Errno 28] No space left on device

In [27]:
prompt = "Suppose you are a content creator and want to generate compelling titles and descriptions for your YouTube videos automatically. Video theme is gym excercise."
# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=200)

In [28]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'user\nSuppose you are a content creator and want to generate compelling titles and descriptions for your YouTube videos automatically. Video theme is gym excercise.\n'

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `ORPOTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.